## Synthetic Data

In [1]:
import numpy as np
from scipy.linalg import kron
import matplotlib.pyplot as plt

def rbf_kernel(x, y, sigma=1.0):
    """Compute the RBF kernel between two high-dimensional vectors."""
    distance_squared = np.sum((x - y) ** 2)
    return np.exp(-distance_squared / (2 * sigma ** 2))

# Create the covariance matrices
def create_cov_matrices(X, H, kernel_func):
    K_X = np.array([[kernel_func(x_i, x_j) for x_j in X] for x_i in X])
    K_H = np.array([[kernel_func(h_i, h_j) for h_j in H] for h_i in H])
    return K_X, K_H

# Generate samples
def sample_gp(X, H, kernel_func):
    K_X, K_H = create_cov_matrices(X, H, kernel_func)
    K = kron(K_H, K_X)
    mean = np.zeros(len(X) * len(H))
    return np.random.multivariate_normal(mean, K).reshape(len(H), len(X))

# Define input space and latent space
X = np.linspace(-100, 100, 100)                             # N=100
H = np.random.multivariate_normal([0, 0], np.eye(2), (20,)) # D=20
# Sample from the GP
noise_free_samples = sample_gp(X, H, rbf_kernel)
eps = 1e-4

# Sample with noise
Y = noise_free_samples.T + np.random.randn(noise_free_samples.T.shape[0],noise_free_samples.T.shape[1]) * eps # N * D

# Reshape X
X = X.reshape(-1,1)

# Plot the samples
'''
for i in range(Y.shape[1]):
    plt.plot(X, noise_free_samples[i], label=f"Output {i+1}")
plt.legend()
plt.title("noise_free_samples from LVMOGP")
plt.show()
'''

'\nfor i in range(Y.shape[1]):\n    plt.plot(X, noise_free_samples[i], label=f"Output {i+1}")\nplt.legend()\nplt.title("noise_free_samples from LVMOGP")\nplt.show()\n'

1. GPy/__init__ 中注释了from numpy.testing import Tester

In [2]:
import GPy as GPy
from GPy.models.gp_multiout_regression import GPMultioutRegression

MyLVMOGP = GPMultioutRegression(X=X, Y=Y, Xr_dim=2, num_inducing=(10,3), init='GP', name='LVMODP_v1')

Intializing with GP...
Fit Sparse GP...
Fit BGPLVM...
Done.


In [3]:
MyLVMOGP.parameter_names()

['inducing inputs',
 'rbf.variance',
 'rbf.lengthscale',
 'Gaussian_noise.variance',
 'kern_row.variance',
 'kern_row.lengthscale',
 'Xr.mean',
 'Xr.variance',
 'Zr',
 'qU_mean',
 'qU_var_col_W',
 'qU_var_col_diag',
 'qU_var_row_W',
 'qU_var_row_diag']

In [20]:
MyLVMOGP

In [ ]:
MyLVMOGP.optimization_runs